<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.0.9/mpnn/examples/afdesign_and_proteinmpnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AfDesign + ProteinMPNN (v1.0.9)
Backprop through AlphaFold for protein design.

**WARNING**
1.   This notebook is in active development and was designed for demonstration purposes only.
2.   Using AfDesign as the only "loss" function for design might be a bad idea, you may find adversarial sequences (aka. sequences that trick AlphaFold). To avoid this problem, we couple it with ProteinMPNN.

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  # get code
  pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.0.9
  # for debugging
  ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign
  # download params
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar | tar x -C params
fi

In [ ]:
#@title import libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign.af import mk_af_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model
from IPython.display import HTML
from google.colab import files
import numpy as np
from scipy.special import softmax

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

from colabdesign.af.alphafold.common import residue_constants
def af2mpnn(self, use_seq=False, use_aux=False, get_best=True):
  atom_idx = tuple(residue_constants.atom_order[k] for k in ["N","CA","C","O"])
  if use_aux:
    aux = self._tmp["best"]["aux"] if (get_best and "aux" in self._tmp["best"]) else self.aux
    X = aux["atom_positions"][:,atom_idx]
    mask = aux["atom_mask"][:,1]
  else:
    X = self._inputs["batch"]["all_atom_positions"][:,atom_idx]
    mask = self._inputs["batch"]["all_atom_mask"][:,1]
  inputs ={"X":X[None],
           "mask":mask[None],
           "residue_idx":self._inputs["residue_index"][None],
           "chain_idx":self._inputs["asym_id"][None],
           "key":self.key()}  
  if use_seq:
    inputs["S"] = aux["aatype"][None] if use_aux else self._inputs["batch"]["aatype"][None]
  return inputs

# fixed backbone design (fixbb)
For a given protein backbone, generate/design a new sequence that AlphaFold thinks folds into that conformation. 

In [ ]:
clear_mem()
af_model = mk_af_model(protocol="fixbb")
af_model.prep_inputs(pdb_filename=get_pdb("1TEN"), chain="A")

print("length",  af_model._len)
print("weights", af_model.opt["weights"])

In [ ]:
# get unconditional probabilities from mpnn
mpnn_model = mk_mpnn_model()
mpnn_inputs = af2mpnn(af_model)
seq_logits = mpnn_model.get_logits(**mpnn_inputs)
seq_logits = np.array(seq_logits)[0,:,:20]

In [ ]:
af_model.restart()
af_model.set_seq(seq=softmax(seq_logits,-1), bias=seq_logits)
af_model.design_3stage(100,100,10)

In [ ]:
af_model.plot_traj()  

In [ ]:
af_model.save_pdb(f"{af_model.protocol}.pdb")
af_model.plot_pdb()

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.get_seqs()

# hallucination
For a given length, generate/hallucinate a protein sequence that AlphaFold thinks folds into a well structured protein (high plddt, low pae, many contacts).

In [ ]:
clear_mem()
af_model = mk_af_model(protocol="hallucination")
af_model.prep_inputs(length=100)

print("length",af_model._len)
print("weights",af_model.opt["weights"])

In [ ]:
# pre-design with gumbel initialization and softmax activation
af_model.restart(mode="gumbel")
af_model.design_soft(50)

# get unconditional probabilities from mpnn
mpnn_model = mk_mpnn_model()
mpnn_inputs = af2mpnn(af_model, use_aux=True)
seq_logits = mpnn_model.get_logits(**mpnn_inputs)
seq_logits = np.array(seq_logits)[0,:,:20]

# three stage design  
af_model.set_seq(seq=softmax(seq_logits,-1), bias=seq_logits)
af_model.design_3stage(50,50,10)

In [ ]:
seq_logits = mpnn_model.get_logits(**af2mpnn(af_model, use_aux=True))
seq_logits = np.array(seq_logits)[0,:,:20]

# three stage design  
af_model.set_seq(seq=softmax(seq_logits,-1), bias=seq_logits)
af_model.design_3stage(50,50,10)

In [ ]:
af_model.save_pdb(f"{af_model.protocol}.pdb")
af_model.plot_pdb()

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.get_seqs()

# binder hallucination
For a given protein target and protein binder length, generate/hallucinate a protein binder sequence AlphaFold thinks will bind to the target structure.
To do this, we minimize PAE and maximize number of contacts at the interface and within the binder, and we maximize pLDDT of the binder.

In [ ]:
clear_mem()
af_model = mk_af_model(protocol="binder")
af_model.prep_inputs(pdb_filename=get_pdb("4MZK"), chain="A", binder_len=19)

print("target_length",af_model._target_len)
print("binder_length",af_model._binder_len)
print("weights",af_model.opt["weights"])

In [ ]:
af_model.restart()
af_model.design_logits(50, save_best=True)

In [ ]:
mpnn_model = mk_mpnn_model()
mpnn_inputs = af2mpnn(af_model, use_aux=True, use_seq=True)

# get unconditional logits for binder
mpnn_inputs["S"][:,af_model._target_len:] = -1
mpnn_inputs["decoding_order"] = np.append(np.arange(af_model._target_len),
                                          np.full(af_model._binder_len,-1))[None]

seq_logits = mpnn_model.get_logits(**mpnn_inputs)
seq_logits = np.array(seq_logits)[0,af_model._target_len:,:20]

In [ ]:
af_model.restart()
af_model.set_seq(seq=softmax(seq_logits,-1), bias=seq_logits)
af_model.design_pssm_semigreedy(0,seq_logits=af_model.opt["bias"])

In [ ]:
af_model.save_pdb(f"{af_model.protocol}.pdb")
af_model.plot_pdb()

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.get_seqs()